In this tutorial, we are going to demonstrate some basic tasks in graph learning. In general, many of the graph learning problems can fall into the following categories:

* node classification: assign a label to a node.
* link prediction: predict the existence of an edge between two nodes.
* graph classification: assign a label to a graph.

There are many real-world applications that can be formulated as one of the graph problems. In fraud detection, we want to predict if a user is a malicious user; we basically assign a binary label to a user. In community detection, we want to predict which community a node belongs to. In recommendation, we want to predict if a user will purchase an item. If he/she does, there is an edge between the user and the item. Thus, recommendation is a link prediction task. In drug discovery, we want to predict the property of a molecule, which can be considered as a graph.

DGL works with different deep learning frameworks. In this tutorial, we show how DGL works with Pytorch. Thus, let's first load Pytorch.

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

We now load DGL. When we load DGL, we need to set the DGL backend. The DGL backend is one of the deep learning frameworks. Currently, DGL has Pytorch and MXNet backends. Because this tutorial develops models in Pytorch, we have to set the DGL backend to Pytorch.

In [ ]:
import dgl
from dgl import DGLGraph
from dgl.data import citegrh
from dgl.nn.pytorch import conv

# Load Pytorch as backend
dgl.load_backend('pytorch')

When we apply GNN to a graph problem, we typically use GNN to compute meaningful node embeddings and then use the node embeddings to perform the downstream task. Thus, the first step is to develop a GNN model to compute node embeddings.

DGL provides an nn module that contains many common GNN layers so that we can develop a GNN model with the layers easily. Later, we'll show how to develop a customized GNN model with DGL's message passing interface.

In this tutorial, we use [GraphSage](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf), one of the first inductive GNN model. Each GraphSage layer performs the following computation on every node $v$ in the graph:

$$h_{N(v)}^k \gets AGGREGATE_k({h_u^{k-1}, \forall u \in N(v)})$$
$$h_v^k \gets \sigma(W^k \cdot CONCAT(h_v^{k-1}, h_{N(v)}^k))$$

, where $N(v)$ is the neighborhood of node $v$ and $k$ is the layer Id.

The GraphSage model stacks multiple layers so that a node $v$ can access neighbors within $k$ hops.

In [ ]:
class GraphSAGE(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGE, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(conv.SAGEConv(in_feats, n_hidden, aggregator_type,
                                         feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(conv.SAGEConv(n_hidden, n_hidden, aggregator_type,
                                             feat_drop=dropout, activation=activation))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h)
        return h

## Load data

In this tutorial, we use a citation network called pubmed for demonstration. DGL has a large collection of built-in datasets. Please see [this doc]() for more information.

A node in the citation network is a paper and an edge represents the citation between two papers. There are 19,717 papers in the dataset and the total number of citations between the papers is 88,651.

When the dataset is loaded into DGL, the network structure is in a [NetworkX]() object. NetworkX is a very popular Python graph library. It provides comprehensive API for graph manipulation and is very useful for preprocessing small graphs.

All other graph data, such as node features and labels, are stored in NumPy tensors. When we load the tensors, we convert them to Pytorch tensors.

In [ ]:
# load and preprocess dataset
data = citegrh.load_pubmed()
features = torch.FloatTensor(data.features)
labels = torch.LongTensor(data.labels)
train_mask = torch.BoolTensor(data.train_mask)
val_mask = torch.BoolTensor(data.val_mask)
test_mask = torch.BoolTensor(data.test_mask)
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = data.graph.number_of_edges()
print("""----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d""" %
          (n_edges, n_classes,
           train_mask.sum().item(),
           val_mask.sum().item(),
           test_mask.sum().item()))

Because the graph structure is stored in a NetworkX object, we can use NetworkX to preprocess the graph. In this example, we remove all self-loops in the graph.

Then we create a DGLGraph from the grpah dataset and convert it to a read-only DGLGraph, which supports more efficient computation. Currently, any sampling API in DGL only works on read-only DGLGraphs.

In [ ]:
g = data.graph
g.remove_edges_from(g.selfloop_edges())
g = DGLGraph(g)
g.readonly()

## Node classification in the semi-supervised setting

The first task is how to perform node classification in a semi-supervised setting. That is, we have the entire graph structure and all node features. We only have labels on some of the nodes. We want to predict the labels on other nodes.

First, we create a 2-layer GraphSage model.

In [ ]:
# Hyperparameters
n_hidden = 64
n_layers = 2
dropout = 0.5
aggregator_type = 'gcn'

gconv_model = GraphSAGE(in_feats,
                        n_hidden,
                        n_layers,
                        F.relu,
                        dropout,
                        aggregator_type)

Now we create the node classification model based on the GraphSage model. The GraphSage model takes a DGLGraph object and node features as input and computes node embeddings as output. With node embeddings, we use a cross entry loss to train the node classification model.

An interested user can try passing any GNN model to the node classification model to compute node embeddings.

In [ ]:
class NodeClassification(nn.Module):
    def __init__(self, gconv_model, n_hidden, n_classes):
        super(NodeClassification, self).__init__()
        self.gconv_model = gconv_model
        self.fc = nn.Linear(n_hidden, n_classes, bias=True)

    def forward(self, g, features):
        embs = self.gconv_model(g, features)
        logits = self.fc(embs)
        return logits

# Node classification task
model = NodeClassification(gconv_model, n_hidden, n_classes)

After defining a model for node classification, we also need to define an evaluation function to evaluate the performance of a trained model.

In [ ]:
def evaluate(model, g, features, labels, test_mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[test_mask]
        test_labels = labels[test_mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == test_labels)
        return correct.item() * 1.0 / len(test_labels)

After defining the model and evaluation function, we can put everything to the training loop.

In [ ]:
# Training hyperparameters
weight_decay = 5e-4
n_epochs = 100
lr = 1e-3

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

loss_fcn = torch.nn.CrossEntropyLoss()

# initialize graph
dur = []
for epoch in range(n_epochs):
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    logits = model(g, features)
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, g, features, labels, val_mask)
    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
            "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur), loss.item(),
                                            acc, n_edges / np.mean(dur) / 1000))

print()
acc = evaluate(model, g, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(acc))



## Link prediction

Link prediction is to predict the existence of an edge between two nodes. In a general case, we consider that an edge connects two similar nodes. Thus, link prediction is basically to find pairs of similar nodes.

Traditionally, one of the most commonly methods for link prediction is SVD. We first use SVD to compute node embeddings and use the embeddings to predict the connection between nodes. The problem with SVD is that it only takes the graph structure into account for link prediction. In many cases, node features provide a lot of information.

When using GNN for link prediction, we can take both the graph structure and node features into consideration when evaluating node similarity.

As the node classification task, we first use GraphSage as the base model to compute node embeddings.

In [ ]:
#Model hyperparameters
n_hidden = 16
n_layers = 1

# create GraphSAGE model
gconv_model = GraphSAGE(in_feats,
                        n_hidden,
                        n_layers,
                        F.relu,
                        dropout,
                        aggregator_type)

Unlike node classification, which uses node labels as training signal, link prediction simply uses the graph structure as the training signal. We consider nodes connected by edges are similar, while nodes not connected by edges are dissimilar.

Thus, the first thing we need to do is to generate positive samples (i.e., pairs of nodes that are connected by edges) and negative edges (i.e., pairs of nodes that are not connected by edges). We should train on each positive sample with multiple negative samples.

DGL provides an edge sampler `EdgeSampler`, which generates positive edge samples and negative edge samples efficiently. We can use this edge sampler to easily generate a batch of positive edges and negative edges. `neg_sample` samples one tenth of the edges in the graph as positive edges and generate `neg_sample_size` negative edges for each positive edge. It generates two subgraphs. The positive subgraph contains all positive edges sampled from the graph `g`, while the negative subgraph contains all negative edges.

In [ ]:
def neg_sample(g, neg_sample_size, edges=None, return_false_neg=True):
    sampler = dgl.contrib.sampling.EdgeSampler(g, batch_size=int(g.number_of_edges()/10),
                                               seed_edges=edges,
                                               neg_sample_size=neg_sample_size,
                                               negative_mode='tail',
                                               shuffle=True,
                                               return_false_neg=return_false_neg)
    sampler = iter(sampler)
    return next(sampler)

After having positive edge subgraphs and negative edge subgraphs, we can now compute the similarity on the positive edge samples and negative edge samples.

In this tutorial, we use cosine similarity to measure the similarity between two nodes.

In [ ]:
def pos_score_func(g, emb):
    pos_src, pos_dst = g.all_edges(order='eid')
    # Get the node Ids in the parent graph.
    pos_src = g.parent_nid[pos_src]
    pos_dst = g.parent_nid[pos_dst]
    # Read the node embeddings of the source nodes and destination nodes.
    pos_heads = emb[pos_src]
    pos_tails = emb[pos_dst]
    # cosine similarity
    return torch.sum(pos_heads * pos_tails, dim=1)

Now we compute the similarity of source nodes and destination nodes on negative edge samples. Because each positive edge is paired with `neg_sample_size`, we reshape the the node embeddings into a 3D tensor of shape (batch_size, neg_sample_size, n_hidden).

In [ ]:
def neg_score_func(g, emb, neg_sample_size):
    neg_src, neg_dst = g.all_edges(order='eid')
    neg_src = g.parent_nid[neg_src]
    neg_dst = g.parent_nid[neg_dst]
    neg_heads = emb[neg_src].reshape(-1, neg_sample_size, emb.shape[1])
    neg_tails = emb[neg_dst].reshape(-1, neg_sample_size, emb.shape[1])
    assert neg_heads.shape[0] == neg_tails.shape[0]
    return torch.sum(neg_heads * neg_tails, dim=2)

Now we put everything together. We first use GraphSage to compute node embeddings. After having the node embeddings, we compute the similarity scores on positive edge samples and negative edge samples. Then we use the following loss function on a positive edge and the corresponding negative edges:

$$L = -log(\sigma(z_u^T z_v)) - Q \cdot E_{v_n\~P_n(v)}(log(\sigma(-z_u^T z_{v_n})))$$

With this loss, training should increase the similarity scores on the positive edges and decrease the similarity scores on negative edges.

In [ ]:
# NCE loss
def NCE_loss(pos_score, neg_score):
    pos_score = F.logsigmoid(pos_score)
    neg_score = F.logsigmoid(-neg_score)
    return -pos_score - torch.sum(neg_score, dim=1)

class LinkPrediction(nn.Module):
    def __init__(self, gconv_model):
        super(LinkPrediction, self).__init__()
        self.gconv_model = gconv_model

    def forward(self, g, features, neg_sample_size):
        emb = self.gconv_model(g, features)
        pos_g, neg_g = neg_sample(g, neg_sample_size, return_false_neg=False)
        pos_score = pos_score_func(pos_g, emb)
        neg_score = neg_score_func(neg_g, emb, neg_sample_size)
        return torch.mean(NCE_loss(pos_score, neg_score))
    
model = LinkPrediction(gconv_model)

Like node classification, we define an evaluation function to evaluate the training result. Ideally, the similarity score of a positive edge should be higher than all negative edges.

In [ ]:
def evaluate(gconv_model, g, features, neg_sample_size):
    gconv_model.eval()
    with torch.no_grad():
        emb = gconv_model(g, features)
        
        pos_g, neg_g = neg_sample(g, neg_sample_size, test_eids, return_false_neg=True)
        pos_score = pos_score_func(pos_g, emb)
        neg_score = neg_score_func(neg_g, emb, neg_sample_size)
        filter_bias = neg_g.edata['false_neg'].reshape(-1, neg_sample_size)

        pos_score = F.logsigmoid(pos_score)
        neg_score = F.logsigmoid(neg_score)
        neg_score -= filter_bias.float()
        pos_score = pos_score.unsqueeze(1)
        rankings = torch.sum(neg_score >= pos_score, dim=1) + 1
        return emb, np.mean(1.0/rankings.cpu().numpy())

Now let's get to the actual training.

We first split the graph into the training set and the testing set. We random sample 80\% edges as the training data and 20\% edges as the testing data. There is no overlap between the training set and the testing set.

In [ ]:
eids = np.random.permutation(g.number_of_edges())
train_eids = eids[:int(len(eids) * 0.8)]
test_eids = eids[int(len(eids) * 0.8):]
train_g = g.edge_subgraph(train_eids, preserve_nodes=True)
test_g = g.edge_subgraph(test_eids, preserve_nodes=True)

The training loop

In [ ]:
# Training hyperparameters
weight_decay = 5e-4
n_epochs = 50
lr = 1e-3
neg_sample_size = 100

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
for epoch in range(n_epochs):
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    loss = model(g, features, neg_sample_size)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    if epoch % 5 == 0:
        _, acc = evaluate(gconv_model, g, features, neg_sample_size)
        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f}"
              .format(epoch, np.mean(dur), loss.item(), acc))
    else:
        print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f}"
              .format(epoch, np.mean(dur), loss.item()))

print()
emb, acc = evaluate(gconv_model, g, features, neg_sample_size)
print("Test Accuracy {:.4f}".format(acc))



In [ ]:
emb